# ultra96-v2   amp-python   A53->R5-0
swig封装c，提供python接口

# 1.虚拟ssh终端调用（效率较低）
初始化虚拟ssh，权限改为root

In [1]:
import sys
sys.path.append('/home/xilinx/.local/lib/python3.6/site-packages')
import paramiko

import time
ssh_client = paramiko.SSHClient()
time.sleep(0.02)
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
time.sleep(0.02)
ssh_client.connect("192.168.3.12", 22, "xilinx","xilinx")
time.sleep(0.02)
yss = ssh_client.invoke_shell()  # 使用伪终端，默认vt100创建交互式
time.sleep(0.02)
yss.send('su -\n')  # 发送su 命令
time.sleep(0.02)
#yss.recv(4096).decode()  # 接收返回
time.sleep(1)
yss.send('xilinx\n')  # 发送root密码
time.sleep(0.02)

In [2]:
yss.recv(65535).decode()  # 接收返回

'Welcome to PYNQ Linux, based on Ubuntu 18.04 (GNU/Linux 4.19.0 aarch64)\r\n\r\nLast login: Thu Nov  5 13:17:38 2020\r\r\nsu -\r\nxilinx@pynq:~$ su -\r\nPassword: \r\nroot@pynq:~# '

In [2]:
from IPython.display import clear_output

启动R5固件

In [3]:
yss.send('modprobe zynqmp_r5_remoteproc\n')
time.sleep(1)
yss.send('echo amp_test2.elf > /sys/class/remoteproc/remoteproc0/firmware\n')
time.sleep(1)
yss.send('echo start > /sys/class/remoteproc/remoteproc0/state\n')
time.sleep(1)

导入amp测试

In [14]:
yss.send('cd /home/xilinx/openamp_python/openamp_final/openamp_test_send\n')
time.sleep(0.5)
yss.send('python3\n')
time.sleep(1)
yss.send('import ampsend\n')
time.sleep(1)

yss.send('ampsend.amp_rpmsg_init()\n')
time.sleep(1)
yss.send('ampsend.amp_send_message(100,100)\n')
time.sleep(1)
#yss.recv(4096).decode()

In [4]:
x=1000000
y=-100000

In [18]:
while True:
    start=time.time()
    a = yss.send('ampsend.amp_send_message('+str(x)+','+str(y)+')\n')
    end=time.time()
    print(end - start)
    time.sleep(0.1)
    clear_output(wait=True)

0.0011548995971679688


KeyboardInterrupt: 

# 2.调用动态链接库.so

调用.so

In [5]:
import ctypes
so = ctypes.CDLL('/home/xilinx/openamp_python/openamp_final/openamp_test_send/build/lib.linux-aarch64-3.6/_ampsend.cpython-36m-aarch64-linux-gnu.so')

启动amp传输测试

In [6]:
so.amp_rpmsg_init()

0

In [7]:
import time
while True:
    start=time.time()
    a = so.amp_send_message(x,y)
    end=time.time()
    print(end - start)
    time.sleep(0.1)
    clear_output(wait=True)

0.0002434253692626953


KeyboardInterrupt: 

# 3.python import module（效率最高）

导入库

In [1]:
import sys
sys.path.append('/home/xilinx/.local/lib/python3.6/site-packages')
sys.path.append('/home/xilinx/openamp_python/openamp_final/openamp_test_send/build/lib.linux-aarch64-3.6')
import paramiko
import time
import _ampsend as amp
from IPython.display import clear_output

虚拟串口改root权限，运行r5固件

In [2]:
ssh_client = paramiko.SSHClient()
time.sleep(0.02)
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
time.sleep(0.02)
ssh_client.connect("192.168.3.12", 22, "xilinx","xilinx")
time.sleep(0.02)
yss = ssh_client.invoke_shell()  # 使用伪终端，默认vt100创建交互式
time.sleep(0.02)
yss.send('su -\n')  # 发送su 命令
time.sleep(0.02)
#yss.recv(4096).decode()  # 接收返回
time.sleep(1)
yss.send('xilinx\n')  # 发送root密码
time.sleep(0.02)

In [3]:
yss.send('modprobe zynqmp_r5_remoteproc\n')
time.sleep(1)
yss.send('echo amp_test2.elf > /sys/class/remoteproc/remoteproc0/firmware\n')
time.sleep(1)
yss.send('echo start > /sys/class/remoteproc/remoteproc0/state\n')
time.sleep(1)

启动amp传输测试

In [4]:
amp.amp_rpmsg_init()
x=1000000
y=-100000

In [6]:
while True:
    start=time.time()
    a = amp.amp_send_message(x,y)
    end=time.time()
    print(end - start)
    time.sleep(0.1)
    clear_output(wait=True)

7.319450378417969e-05


KeyboardInterrupt: 

注：需要修改一下jupyter权限为root，否则初始化amp有问题，参考：https://blog.csdn.net/donaldsy/article/details/96194346